In [1]:
import sys
print(sys.path)


['/home/stavisa/ufpr-thesis', '/home/stavisa/programs/anaconda3/lib/python310.zip', '/home/stavisa/programs/anaconda3/lib/python3.10', '/home/stavisa/programs/anaconda3/lib/python3.10/lib-dynload', '', '/home/stavisa/programs/anaconda3/lib/python3.10/site-packages', '/home/stavisa/programs/anaconda3/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg', '/home/stavisa/programs/anaconda3/lib/python3.10/site-packages/pybind11-2.13.5-py3.10.egg', '/home/stavisa/programs/anaconda3/lib/python3.10/site-packages/mpmath-1.2.1-py3.10.egg']


In [2]:
import torch
import torch.nn as nn
import numpy as np
import sklearn.metrics
import torch.optim as optim
import dataset
import onehot
import w2v

In [ ]:
# Define the classifier (same as in BertForSequenceClassification)
class W2VClassifier(nn.Module):
    def __init__(self, hidden_size, num_labels, dropout_prob):
        super(W2VClassifier, self).__init__()
        self.dropout = nn.Dropout(dropout_prob)
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, embeddings):
        # Apply dropout
        output = self.dropout(embeddings)
        # Apply classifier (linear layer)
        logits = self.classifier(output)
        return logits


In [ ]:
w2v_embeddings = torch.tensor(w2v_embedding)  # Ensure this matches the hidden_size
labels = torch.tensor([1, 0]) 

In [ ]:
# Define the function (as provided)
def calculate_metric_with_sklearn(predictions: np.ndarray, labels: np.ndarray):
    valid_predictions = predictions
    valid_labels = labels
    return {
        "accuracy": sklearn.metrics.accuracy_score(valid_labels, valid_predictions),
        "f1": sklearn.metrics.f1_score(
            valid_labels, valid_predictions, average="macro", zero_division=0
        ),
        "matthews_correlation": sklearn.metrics.matthews_corrcoef(
            valid_labels, valid_predictions
        ),
        "precision": sklearn.metrics.precision_score(
            valid_labels, valid_predictions, average="macro", zero_division=0
        ),
        "recall": sklearn.metrics.recall_score(
            valid_labels, valid_predictions, average="macro", zero_division=0
        ),
    }


In [ ]:
# Hyperparameters
learning_rate = 0.001
num_epochs = 10
batch_size = 32

# Parameters
hidden_size = 768  # Adjust this based on your W2V embedding size (must match BERT's hidden size)
num_labels = 2     # Number of classes (e.g., for binary classification)
dropout_prob = 0.1  # Same dropout as in the original BERT classifier


# Initialize the model, optimizer, and loss function
model = W2VClassifier(hidden_size, num_labels, dropout_prob)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

In [18]:
input_path = 'train.csv'
data, labels = onehot.process_csv(input_path)

NameError: name 'one_hot1' is not defined

In [ ]:
train_dataset = MyDataset(data, labels)

# Create a DataLoader
dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)

In [ ]:

model.train()  # Set the model to training mode
# Training loop
for epoch in range(num_epochs):

    total_loss = 0
    
    # Iterate through batches (assumed you have a function for batching)
    for batch in train_data_loader:  # Replace with your DataLoader
        inputs, labels = batch  # Get inputs and labels

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        logits = model(inputs)

        # Compute loss
        loss = loss_function(logits, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(train_data_loader):.4f}")

# After training, you can evaluate the model on the validation set


In [ ]:
all_predictions = []
all_labels = []

model.eval()  # Set the model to evaluation mode
for epoch in range(num_epochs):

  for inputs, labels in validation_data_loader: 

    # Get the logits from the classifier
    logits = model(inputs)

    # Predicted classes (index of the max logits)
    predicted_classes = torch.argmax(logits, dim=-1)

    all_predictions.extend(predicted_classes.numpy())
    all_labels.extend(labels.numpy())


all_predictions = np.array(all_predictions)
all_labels = np.array(all_labels)

# Call the function to calculate metrics
metrics = calculate_metric_with_sklearn(all_predictions, all_labels)

# Output the calculated metrics
print(metrics)